# LLM as Player

Here we use an LLM to make decisions.

- This example includes utilities for estimating the cost of LLM calls.
- To set expectations: the prompt and the information provided to the LLM are intentionally simple and could be expanded. Still, it’s a solid starting point, and the **cost summary at the end of the notebook** provides a sense of what it can cost to play a game.
- The notebook contains the full log history of a 10 hand long game played using **gpt-5-nano against 3 simple bots**. **The model untimately won the game** with a few good moves and a number of questionable decisions as well.

In [1]:
#pip install openai ollama python-dotenv

In [2]:
import logging, sys, os

from openai import OpenAI
from openai.types.responses.response_usage import ResponseUsage
from dotenv import load_dotenv

from maverick import (
    Game,
    Player,
    ActionType,
    PlayerLike,
    PlayerAction,
    PlayerState,
)
from maverick.players import FoldBot, CallBot, AggressiveBot
from maverick.utils import estimate_holding_strength, find_highest_scoring_hand

_ = load_dotenv(override=True)

In [3]:
from collections import defaultdict

KNOWN_LLM_PRICES = {
    "gpt-5.2":    {"input": 1.75, "cached": 0.175, "output": 14.00},
    "gpt-5.1":    {"input": 1.25, "cached": 0.125, "output": 10.00},
    "gpt-5":      {"input": 1.25, "cached": 0.125, "output": 10.00},
    "gpt-5-mini": {"input": 0.25, "cached": 0.025, "output": 2.00},
    "gpt-5-nano": {"input": 0.05, "cached": 0.005, "output": 0.40},
}

LLM_PRICES = defaultdict(lambda: {"input": 0.0, "cached": 0.0, "output": 0.0}, KNOWN_LLM_PRICES)


def llm_call_cost(
    *,
    input_tokens: int,
    output_tokens: int,
    model: str,
    cached_input_tokens: int = 0,
) -> float:
    p = LLM_PRICES[model]
    uncached_input = max(0, input_tokens - cached_input_tokens)
    total_cost =  (
        uncached_input / 1_000_000 * p["input"]
        + cached_input_tokens / 1_000_000 * p["cached"]
        + output_tokens / 1_000_000 * p["output"]
    )
    return total_cost

In [4]:
SYSTEM_MESSAGE = """
You are an expert Texas Hold'em poker agent.

You will be given the current game state and a list of valid actions.
Your task: choose exactly one valid action and (if required) a numeric amount.

Action semantics:
- FOLD: provide no `amount`.
- CHECK: provide no `amount`.
- CALL: provide no `amount` (the engine infers call cost).
- BET: provide `amount` = the amount you transfer from your stack to the pot.
  The amount MUST be >= min_bet_amount.
- RAISE: provide `amount` = the amount you transfer from your stack to the pot.
  The amount MUST be >= min_raise_amount.
  
Card notation:
- Cards use standard short codes: Ah=Ace of hearts, Td=Ten of diamonds, 2c=Two of clubs.
- Ranks: 2-9,T,J,Q,K,A. Suits: h,d,c,s.
  
Glossary of terms:
- equity_vs_field = probability that your hand wins against all remaining players,
  estimated via Monte-Carlo simulation.
- best_current_hand = the strongest 5-card hand you can make now with your hole cards
  and the community cards.
- best_hand_type = the type of your best current hand (e.g., "FLUSH", "STRAIGHT").

Output requirements:
- Output MUST conform to the PlayerAction schema.
- Do NOT output explanations or any extra text.
- Never choose an action not present in valid_actions.
"""


class LLMPlayer(Player):
    
    def __init__(self, *args, api_key:str, model:str, **kwargs):
        super().__init__(*args, **kwargs)
        self.model = model
        self._reset_usage()
        if "_usage" in kwargs and kwargs["_usage"] is not None:
            self._usage = kwargs["_usage"]
        self._set_up_llm_client(api_key=api_key)
            
    def _set_up_llm_client(self, api_key: str) -> None:
        """Set up the LLM client with the provided API key."""
        if api_key.lower() == "ollama":
            OLLAMA_BASE_URL = "http://localhost:11434/v1"
            self.llm = OpenAI(base_url=OLLAMA_BASE_URL, api_key='ollama')
        else:
            self.llm = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))
        
    def to_dict(self) -> dict:
        base = super().to_dict()
        base["_usage"] = self._usage
        return base
        
    def _reset_usage(self) -> None:
        """Reset the usage statistics for this player."""
        self._usage = {
            "input_tokens": 0,
            "output_tokens": 0,
            "total_tokens": 0,
            "input_tokens_details": {
                "cached_tokens": 0,
            },
            "output_tokens_details": {
                "reasoning_tokens": 0
            }
        }
    
    def _accumulate_usage(self, usage: ResponseUsage) -> None:
        self._usage["input_tokens"] += usage.input_tokens
        self._usage["output_tokens"] += usage.output_tokens
        self._usage["total_tokens"] += usage.total_tokens
        if usage.input_tokens_details:
            self._usage["input_tokens_details"]["cached_tokens"] += usage.input_tokens_details.cached_tokens
        if usage.output_tokens_details:
            self._usage["output_tokens_details"]["reasoning_tokens"] += usage.output_tokens_details.reasoning_tokens
        
    def _total_call_cost(self) -> float:
        """Calculate the total cost of all OpenAI calls made by this player."""
        input_tokens = self._usage["input_tokens"]
        output_tokens = self._usage["output_tokens"]
        cached_input_tokens = self._usage["input_tokens_details"]["cached_tokens"]
        
        return llm_call_cost(
            input_tokens=input_tokens,
            output_tokens=output_tokens,
            model=self.model,
            cached_input_tokens=cached_input_tokens,
        )
    
    def decide_action(
        self,
        *,
        game: Game,
        valid_actions: list["ActionType"],
        min_raise_amount: int,
        call_amount: int,
        min_bet_amount: int,
    ) -> "PlayerAction":

        # ---------- Debug prints (optional) ----------
        print(f"\n{self.name} is requested to take action.")
        print("     Current street:", game.state.street.name)
        print("     Button position:", game.state.button_position)
        print("     Pot:", game.state.pot)
        print("     Current bet:", game.state.current_bet)
        print("     Small blind:", game.state.small_blind)
        print("     Big blind:", game.state.big_blind)
        print("     Community cards:", [str(c) for c in game.state.community_cards])
        print("     Player stack:", self.state.stack)
        print("     Player current bet:", self.state.current_bet)
        print("     Amount to call:", call_amount)
        print("     Private cards:", [str(c) for c in self.state.holding.cards])
        print("     Valid actions:", [v.name for v in valid_actions])
        print("     Min raise amount:", min_raise_amount)
        print("     Min bet amount:", min_bet_amount)

        private_cards = self.state.holding.cards
        community_cards = game.state.community_cards

        # ---------- Hand evaluation (your existing utilities) ----------
        n_private_required=game.rules.showdown.hole_cards_required
        (
            strongest_hand, 
            strongest_hand_type, 
            strongest_hand_score
        ) = find_highest_scoring_hand(
            private_cards, 
            community_cards, 
            n_private=n_private_required
        )

        hand_prob = estimate_holding_strength(
            private_cards,
            community_cards=community_cards,
            n_simulations=1000,
            n_private=n_private_required,
            n_players=len(game.state.get_players_in_hand()),
        )

        print("     Best hand type:", strongest_hand_type.name)
        print("     Best hand score:", strongest_hand_score)
        print("     Equity (MC):", hand_prob)

        # ---------- Prompt (structured, minimal, unambiguous) ----------
        player_message = f"""
STATE (chip amounts are integers):

player:
  id: {self.id}
  name: {self.name}
  stack: {int(self.state.stack)}
  current_bet: {int(self.state.current_bet)}
  amount_to_call: {call_amount}
  hole_cards: {', '.join(card.code() for card in private_cards)}

table:
  street: {game.state.street.name}
  community_cards: {', '.join(card.code() for card in community_cards)}
  pot: {int(game.state.pot)}
  current_bet: {int(game.state.current_bet)}
  blinds: [{int(game.state.small_blind)}, {int(game.state.big_blind)}]
  players_in_hand: {len(game.state.get_players_in_hand())}

constraints:
  valid_actions: {[v.name for v in valid_actions]}
  min_raise_amount: {min_raise_amount}
  min_bet_amount: {min_bet_amount}

hand_eval:
  best_current_hand: {strongest_hand}
  best_hand_type: {strongest_hand_type.name}
  equity_vs_field: {hand_prob:.3f}

Choose your next action now.
If action is BET or RAISE, include integer amount.
Remember: for RAISE, amount must be >= min_raise_amount.
Remember: for BET, amount must be >= min_bet_amount.
"""

        # ---------- Call the model ----------
        try:
            response = self.llm.responses.parse(
                model=self.model,
                input=[
                    {"role": "system", "content": SYSTEM_MESSAGE},
                    {"role": "user", "content": player_message},
                ],
                text_format=PlayerAction,
            )
            self._accumulate_usage(response.usage)
            action: PlayerAction = response.output_parsed
            print(f"     LLMPlayer raw action output: {action}")  
        except Exception as e:
            logging.error(f"LLMPlayer {self.id} failed to decide action: {e}")
            action = PlayerAction(player_id=self.id, action_type=ActionType.FOLD)

        # ---------- Post-parse validation & self-heal ----------
        valid_set = set(valid_actions)
        if action.action_type not in valid_set:
            # safest fallback: CHECK if possible else FOLD
            logging.error("LLMPlayer produced invalid action type, applying fallback.")
            fallback = ActionType.CHECK if ActionType.CHECK in valid_set else ActionType.FOLD
            action = PlayerAction(player_id=self.id, action_type=fallback)

        needs_amount = action.action_type in {ActionType.BET, ActionType.RAISE}
        if needs_amount:
            # Missing amount -> pick a conservative default
            if getattr(action, "amount", None) is None:
                logging.error("LLMPlayer missing amount for bet/raise, applying default.")
                if action.action_type == ActionType.RAISE:
                    action.amount = min_raise_amount
                else:
                    action.amount = min_bet_amount
            # Normalize to int and clamp
            action.amount = int(action.amount)

            # Enforce min raise amount
            if action.action_type == ActionType.RAISE and action.amount < min_raise_amount:
                action.amount = min_raise_amount
                
            # Enforce min bet amount
            if action.action_type == ActionType.BET and action.amount < min_bet_amount:
                action.amount = min_bet_amount

            # No negative or zero bet/raise
            if action.amount <= 0:
                logging.error("LLMPlayer produced non-positive amount for bet/raise, applying fallback.")
                fallback = ActionType.CHECK if ActionType.CHECK in valid_set else ActionType.FOLD
                action = PlayerAction(player_id=self.id, action_type=fallback)

            # Can't wager more than stack in this action (if your engine allows all-in as exact stack, keep this)
            if action.amount > int(self.state.stack):
                logging.error("LLMPlayer produced amount exceeding stack, clamping to stack.")
                action.amount = int(self.state.stack)

        else:
            # Strip stray amount for non-sizing actions
            if hasattr(action, "amount") and action.amount is not None:
                logging.error("LLMPlayer produced amount for non-sizing action, stripping.")
                action.amount = None

        print(f"     Player decides to {action.action_type.name}"
              f"{'' if getattr(action, 'amount', None) is None else f' ({action.amount})'}.\n")
        
        sys.stdout.flush()
        return action


In [ ]:
llm_player = LLMPlayer(
    name="LLM", 
    state=PlayerState(stack=1000),
    api_key=os.getenv('OPENAI_API_KEY'),
    model="gpt-5-nano",
)

# llm_player = LLMPlayer(
#     name="LLM", 
#     state=PlayerState(stack=1000),
#     api_key="ollama",
#     model="deepseek-r1:8b",
# )

response = llm_player.llm.responses.parse(
    model=llm_player.model, 
    input=[{"role": "user", "content": "Tell me a poker joke."}]
)
response.output_text, response.usage.total_tokens

('Yes, I’m ready. Want to play Texas Hold’em? We can do a quick heads-up round with 1000 chips each, blinds 1/2. If that works, say “start” and I’ll deal the first hand. You’ll respond with actions like: check, bet <amount>, call, raise <amount>, or fold.',
 1691)

In [6]:
llm_call_cost(
    input_tokens=response.usage.input_tokens,
    output_tokens=response.usage.output_tokens,
    model=llm_player.model,
    cached_input_tokens=response.usage.input_tokens_details.cached_tokens if response.usage.input_tokens_details else 0,
)

0.0006718500000000001

In [7]:
# Configure logging such that we only get the log messages of the game
logging.basicConfig(level=logging.DEBUG, format="%(name)s: %(message)s")
logging.getLogger().setLevel(logging.WARNING)  # Set root logger to WARNING
logging.getLogger("maverick").setLevel(logging.DEBUG)  # Only maverick logs at DEBUG

# Create game with blinds
game = Game(small_blind=10, big_blind=20, max_hands=10)

# Create and add players with different strategies
players: list[PlayerLike] = [
    CallBot(name="CallBot", state=PlayerState(stack=1000)),
    AggressiveBot(name="AggroBot", state=PlayerState(stack=1000)),
    FoldBot(name="FoldBot", state=PlayerState(stack=1000)),
    llm_player,
]

for player in players:
    game.add_player(player)

game.start()

maverick: Player CallBot joined the game.
maverick: Player AggroBot joined the game.
maverick: Player FoldBot joined the game.
maverick: Player LLM joined the game.
maverick: Game started.

maverick: ============================== Hand 1 ==============================

maverick: PRE_FLOP | Dealing hole cards. Button: FoldBot
maverick: PRE_FLOP | Posting small blind of 10 by player LLM. Remaining stack: 990
maverick: PRE_FLOP | Posting big blind of 20 by player CallBot. Remaining stack: 980
maverick: PRE_FLOP | Player AggroBot raises by 40 chips to total bet 40. Remaining stack: 960.
maverick: PRE_FLOP | Current pot: 70 | Current bet: 40
maverick: PRE_FLOP | Player FoldBot folds.
maverick: PRE_FLOP | Current pot: 70 | Current bet: 40



LLM is requested to take action.
     Current street: PRE_FLOP
     Button position: 2
     Pot: 70
     Current bet: 40
     Small blind: 10
     Big blind: 20
     Community cards: []
     Player stack: 990
     Player current bet: 10
     Amount to call: 30
     Private cards: ['3♠', 'K♥']
     Valid actions: ['FOLD', 'CALL', 'RAISE', 'ALL_IN']
     Min raise amount: 50
     Min bet amount: 20
     Best hand type: HIGH_CARD
     Best hand score: 100.1303
     Equity (MC): 0.315
     LLMPlayer raw action output: player_id='019b39c67442454a866e029bee296b8c' action_type=<ActionType.CALL: 3> amount=None
     Player decides to CALL.



maverick: PRE_FLOP | Player LLM calls with amount 30. Remaining stack: 960.
maverick: PRE_FLOP | Current pot: 100 | Current bet: 40
maverick: PRE_FLOP | Player CallBot calls with amount 20. Remaining stack: 960.
maverick: PRE_FLOP | Current pot: 120 | Current bet: 40
maverick: PRE_FLOP | Betting round complete

maverick: FLOP | Dealt flop. Community cards: ['9♠', '5♣', '8♥']



LLM is requested to take action.
     Current street: FLOP
     Button position: 2
     Pot: 120
     Current bet: 0
     Small blind: 10
     Big blind: 20
     Community cards: ['9♠', '5♣', '8♥']
     Player stack: 960
     Player current bet: 0
     Amount to call: 0
     Private cards: ['3♠', 'K♥']
     Valid actions: ['FOLD', 'CHECK', 'BET', 'ALL_IN']
     Min raise amount: 0
     Min bet amount: 20
     Best hand type: HIGH_CARD
     Best hand score: 100.1309080503
     Equity (MC): 0.165
     LLMPlayer raw action output: player_id='019b39c67442454a866e029bee296b8c' action_type=<ActionType.FOLD: 1> amount=None
     Player decides to FOLD.



maverick: FLOP | Player LLM folds.
maverick: FLOP | Current pot: 120 | Current bet: 0
maverick: FLOP | Player CallBot checks.
maverick: FLOP | Current pot: 120 | Current bet: 0
maverick: FLOP | Player AggroBot bets amount 40. Remaining stack: 920.
maverick: FLOP | Current pot: 160 | Current bet: 40
maverick: FLOP | Player CallBot calls with amount 40. Remaining stack: 920.
maverick: FLOP | Current pot: 200 | Current bet: 40
maverick: FLOP | Betting round complete

maverick: TURN | Dealt turn. Community cards: ['9♠', '5♣', '8♥', '3♣']
maverick: TURN | Player CallBot checks.
maverick: TURN | Current pot: 200 | Current bet: 0
maverick: TURN | Player AggroBot bets amount 40. Remaining stack: 880.
maverick: TURN | Current pot: 240 | Current bet: 40
maverick: TURN | Player CallBot calls with amount 40. Remaining stack: 880.
maverick: TURN | Current pot: 280 | Current bet: 40
maverick: TURN | Betting round complete

maverick: RIVER | Dealt river. Community cards: ['9♠', '5♣', '8♥', '3♣', 'Q♦'

...

root: LLMPlayer produced amount for non-sizing action, stripping.


     LLMPlayer raw action output: player_id='019b39c67442454a866e029bee296b8c' action_type=<ActionType.ALL_IN: 6> amount=1550
     Player decides to ALL_IN.



maverick: RIVER | Player LLM goes all-in with 1550 chips.
maverick: RIVER | Current pot: 2110 | Current bet: 1550
maverick: RIVER | Player AggroBot calls with amount 950. Remaining stack: 0.
maverick: RIVER | Current pot: 3060 | Current bet: 1550
maverick: RIVER | Betting round complete

maverick: SHOWDOWN | Player AggroBot has holding 3♠ 2♣ at showdown,
maverick: SHOWDOWN | Player AggroBot has hand PAIR with cards ['2♣', '9♣', 'A♣', '2♥', '10♥'] (score: 202.14101)
maverick: SHOWDOWN | Player LLM has holding 4♥ A♥ at showdown,
maverick: SHOWDOWN | Player LLM has hand FLUSH with cards ['4♥', 'A♥', '2♥', '10♥', '6♥'] (score: 600.14101)
maverick: SHOWDOWN | Player LLM wins 3060 from the pot.
maverick: SHOWDOWN | Showdown complete

maverick: Hand ended

maverick: ============================== Hand 8 ==============================

maverick: PRE_FLOP | Dealing hole cards. Button: FoldBot
maverick: PRE_FLOP | Posting small blind of 10 by player FoldBot. Remaining stack: 930
maverick: PRE_FL


LLM is requested to take action.
     Current street: PRE_FLOP
     Button position: 1
     Pot: 30
     Current bet: 20
     Small blind: 10
     Big blind: 20
     Community cards: []
     Player stack: 3060
     Player current bet: 10
     Amount to call: 10
     Private cards: ['7♠', '7♣']
     Valid actions: ['FOLD', 'CALL', 'RAISE', 'ALL_IN']
     Min raise amount: 30
     Min bet amount: 20
     Best hand type: PAIR
     Best hand score: 207
     Equity (MC): 0.687
     LLMPlayer raw action output: player_id='019b39c67442454a866e029bee296b8c' action_type=<ActionType.CALL: 3> amount=None
     Player decides to CALL.



maverick: PRE_FLOP | Player LLM calls with amount 10. Remaining stack: 3050.
maverick: PRE_FLOP | Current pot: 40 | Current bet: 20
maverick: PRE_FLOP | Player FoldBot folds.
maverick: PRE_FLOP | Current pot: 40 | Current bet: 20
maverick: PRE_FLOP | Betting round complete

maverick: SHOWDOWN | Player LLM wins 40 from the pot.
maverick: SHOWDOWN | Showdown complete

maverick: Hand ended

maverick: ============================== Hand 10 ==============================

maverick: PRE_FLOP | Dealing hole cards. Button: FoldBot
maverick: PRE_FLOP | Posting small blind of 10 by player FoldBot. Remaining stack: 900
maverick: PRE_FLOP | Posting big blind of 20 by player LLM. Remaining stack: 3070
maverick: PRE_FLOP | Player FoldBot folds.
maverick: PRE_FLOP | Current pot: 30 | Current bet: 20
maverick: PRE_FLOP | Betting round complete

maverick: SHOWDOWN | Player LLM wins 30 from the pot.
maverick: SHOWDOWN | Showdown complete

maverick: Hand ended

maverick: Reached maximum number of hands, 

In [8]:
for player in players:
    print(f"{player.name} - Stack: {player.state.stack}")

CallBot - Stack: 0
AggroBot - Stack: 0
FoldBot - Stack: 900
LLM - Stack: 3100


In [10]:
token_usage = llm_player._usage

input_tokens = token_usage["input_tokens"]
output_tokens = token_usage["output_tokens"]
cached_input_tokens = token_usage["input_tokens_details"]["cached_tokens"]

print(f"{llm_player.model} total token usage: ${llm_player._total_call_cost():.6f}\n")

for model_name in KNOWN_LLM_PRICES:
    if model_name == llm_player.model:
        continue  # already printed
    
    cost = llm_call_cost(
        input_tokens=input_tokens,
        output_tokens=output_tokens,
        model=model_name,
        cached_input_tokens=cached_input_tokens,
    )
    print(f"Total cost on {model_name} would be: ${cost:.6f}")

gpt-5-nano total token usage: $0.048079

Total cost on gpt-5.2 would be: $1.682774
Total cost on gpt-5.1 would be: $1.201981
Total cost on gpt-5 would be: $1.201981
Total cost on gpt-5-mini would be: $0.240396
